# Changes from v9
* Just running CPT codes for the first code that was assigned
* 64% accuracy on top 3
* Resampling and choosing those over 200
* 57% accuracy
* Choosing the top 4 with resampling
* 54% accuracy


# Import the MIMIC data

In [1]:
import gzip
import pandas as pd
import glob

dataset_dictionary = {}

for file_path in glob.glob('.\\Data\\MIMIC Files\*'):
    file_name = file_path.split('\\')[3].split('.')[0]
    with gzip.open(file_path, mode='r') as file:
        dataset_dictionary[file_name] = pd.read_csv(file)

c:\users\amartins\onedrive - intermountain healthcare\python_pycharm_virt_env\.venv\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (4,5,7,11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
c:\users\amartins\onedrive - intermountain healthcare\python_pycharm_virt_env\.venv\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# Join the tables

In [94]:
# Create dataset to join together -----

# Create note_events table -----

# Combine text for each subject and encounter
note_events_base = dataset_dictionary['NOTEEVENTS'][dataset_dictionary['NOTEEVENTS'].loc[:,'CATEGORY'] == 'Discharge summary']
note_events = note_events_base.groupby(['SUBJECT_ID', 'HADM_ID'], as_index=False)['TEXT'].agg(sum)

# Create CPT table -----

cpt_events_base = dataset_dictionary['CPTEVENTS']
cpt_events_base = cpt_events_base[cpt_events_base['TICKET_ID_SEQ'] == 1]
cpt_events_base = cpt_events_base.loc[:, ['SUBJECT_ID','HADM_ID', 'CPT_CD']]
cpt_events = cpt_events_base.drop_duplicates()
cpt_events

# Join the datasets -----

note_cpt = note_events.merge(cpt_events, on = ['SUBJECT_ID','HADM_ID'])
# print(note_cpt.shape, note_events.shape, cpt_events.shape) # (223,150, 4) (52,726, 3) (227,510, 3)

# Filter the data to CPT over 200 samples + Resample

In [95]:
import numpy as np
from sklearn.utils import resample

# Value Counts
print(note_cpt['CPT_CD'].astype(str).value_counts())

# Filter to CPT with over 200 notes
df = note_cpt['CPT_CD'].astype(str).value_counts()
top_200 = list((df[df > 200]).index.values)
note_cpt_4 = note_cpt[note_cpt['CPT_CD'].astype(str).isin(top_200)]

# Resample minority groups -----

# Remove largest group
top_200.remove('99291')
'99291' in top_200

# minority_ls = top_200
minority_ls = ['99223','99222','99254']

minority_df = []
for i in minority_ls:
    test_resampled = resample(note_cpt[note_cpt['CPT_CD'].astype(str) == i], replace=True, n_samples=7860, random_state=123)
    minority_df.append(test_resampled)

minority_df.append(note_cpt[note_cpt['CPT_CD'].astype(str) == '99291'])
new_df = pd.concat(minority_df)

new_df['CPT_CD'] = new_df['CPT_CD'].astype(str)

99291    7860
99223    2851
99222    1736
99254    1242
99255     882
         ... 
32997       1
21510       1
26540       1
35626       1
44310       1
Name: CPT_CD, Length: 707, dtype: int64


# Check for Imbalance

In [105]:
import matplotlib.pyplot as plt

# plt.hist(note_cpt['CPT_CD'].astype(str))
# plt.show()

99222    7860
99291    7860
99223    7860
99254    7860
Name: CPT_CD, dtype: int64

# Filter the data

In [99]:
import string

def clean_data(text_series):
    
    # Replace \n 
    text_series = text_series.str.replace('\\n',' ', regex=True)    

    # Remove dates and locations
    text_series = text_series.str.replace('\[\*\*(.*?)\*\*\]', ' ', regex=True)
    
    # Remove topics
    data = text_series.str.split('([A-Z\s]+:)')
    for row_num, value in enumerate(data):
        text_chunks = [x.strip().replace(':','').replace('\n', '') for x in value]
        for i, x in enumerate(text_chunks):
            if 'MEDICATION' in x or 'SOCIAL HISTORY' in x or 'FAMILY HISTORY' in x:
                text_chunks[i] = ' '
                try:
                    text_chunks[i + 1] = ' '
                except:
                    continue

        text_series.iloc[row_num] = ' '.join(text_chunks)
    
    # Replace punctuation
    text_series = text_series.str.replace('[' + string.punctuation + ']', ' ', regex=True)
    
    # Convert to lowercase 
    text_series = text_series.str.lower()
    
    # Remove all digits
    text_series = text_series.str.replace('\d',' ', regex=True)
    
    return text_series

# Update Text Column

new_df.loc[:, 'TEXT'] = clean_data(new_df['TEXT']).values

# Shuffle the Data

In [100]:
new_df = new_df.sample(n = len(new_df), random_state = 42)

# Split the Data

In [101]:
# Import Packages -----

from nltk.corpus import stopwords

my_stop_words = list(set(stopwords.words('english'))) \
                + ['admission', 'date', 'sex'] \
                + ['needed', 'every', 'seen', 'weeks', 'please', 'ml', 'unit', 'small', 'year', 'old', 'cm', 'non', 'mm', 'however']
                # Got the above from my top 100 most predictive words that I wanted to remove

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# Split the data -----

X_train, X_test, y_train, y_test = train_test_split(new_df['TEXT'].values, new_df['CPT_CD'].astype(str), test_size = .33, random_state = 42)

# Tokenize the data

In [102]:
# Tokenize the data -----

# Import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words=my_stop_words, min_df = 3, max_df = .7, sublinear_tf=True)

# Transform the training data
tfidf_train = tfidf_vectorizer.fit_transform(X_train)

# Transform the test data
tfidf_test = tfidf_vectorizer.transform(X_test)

# Run Naive Bayes

In [103]:
# Use Naive Bayes model -----

from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

nb_classifier = MultinomialNB(alpha=.7)

# Fit and check accuracy
nb_classifier.fit(tfidf_train, y_train)
pred = nb_classifier.predict(tfidf_test)

# Tune NB Model

In [18]:
import numpy as np


def hyperparam_tuning(tfidf_train, y_train, tfidf_test, y_test, nb_classifier):
    for i in np.arange(0,1.1,.1):
        nb_classifier = MultinomialNB()
        nb_classifier.fit(tfidf_train, y_train)
        pred = nb_classifier.predict(tfidf_test)
        print(i)
        print(metrics.accuracy_score(y_test, pred))

hyperparam_tuning(tfidf_train, y_train, tfidf_test, y_test, nb_classifier)  

# Looks like .6-.7 are the best alpha

0.0
0.421076621541547
0.1
0.421076621541547
0.2
0.421076621541547
0.30000000000000004
0.421076621541547
0.4
0.421076621541547
0.5
0.421076621541547
0.6000000000000001
0.421076621541547
0.7000000000000001
0.421076621541547
0.8
0.421076621541547
0.9
0.421076621541547
1.0
0.421076621541547


# Run Logistic Regression

In [38]:
from sklearn.linear_model import LogisticRegression

clf_logist = LogisticRegression(C=.001, random_state = 42, multi_class = 'multinomial', penalty='l2')
clf_logist.fit(tfidf_train, y_train)
logist_pred = clf_logist.predict(tfidf_test)

c:\users\amartins\onedrive - intermountain healthcare\python_pycharm_virt_env\.venv\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# Looking at Feature Names and Weights

In [95]:
# Notes
# sum([np.exp(1)** x for x in nb_classifier.coef_[0]]) # The probability of all the words equals one
# # Taken from here: * https://stackoverflow.com/questions/61586946/how-to-calculate-feature-log-prob-in-the-naive-bayes-multinomialnb

# ------------------------------------------

import numpy as np

def get_feature_rank(tfidf_vectorizer, y_no, nb_classifier):
    
    # Get the feature names
    feature_names = tfidf_vectorizer.get_feature_names()

    # Zip together the first CPT weights with feature names
    feat_with_weights =  sorted(zip(nb_classifier.coef_[y_no], feature_names))
    
    # Print words most responsible for the prediction
#     print('Top 100 \n\n\n\n')
#     top_100_ls = []
    for i in range(100):
        x = feat_with_weights[-i-1]
#         top_100_ls.append(x[1])
#         print(nb_classifier.classes_[y_no], i, round((np.exp(1) ** x[0]),4), x[1])

#     print('\n\n\n\n Bottom 100 \n\n\n\n')
    for i in range(100):
        x = feat_with_weights[i]
#         print(nb_classifier.classes_[y_no], i, round((np.exp(1) ** x[0]),4), x[1])
    
#     min_weight = min([i[0] for i in feat_with_weights])
    
    x = [i[0] for i in feat_with_weights]
    
    median_pred = np.median(x)
          
    return [i[1] for i in feat_with_weights if i[0] <= median_pred] # Minimum weight words
#     return top_100_ls

# Find the least predictive words
def least_pred_words(nb_classifier, tfidf_vectorizer):
    low_wt_stop_ls = []

    for i in range(len(nb_classifier.classes_)):
        low_wt_stop_ls += get_feature_rank(tfidf_vectorizer, i, nb_classifier)

    low_wt_stop_ls = list(set(low_wt_stop_ls))
    return low_wt_stop_ls
    
low_wt_stop_ls = least_pred_words(nb_classifier, tfidf_vectorizer)

# Find top 100 words - doesn't seem to improve the model
def highest_pred_words(nb_classifier, tfidf_vectorizer):
    top_100_ls = []
    for i in range(len(nb_classifier.classes_)):
        top_100_ls += get_feature_rank(tfidf_vectorizer, i, nb_classifier)

    top_100_ls = list(set(top_100_ls))
    return top_100_ls

33442

# Update stop words and tokenize again

In [96]:
my_stop_words += low_wt_stop_ls

tfidf_vectorizer = TfidfVectorizer(stop_words=my_stop_words, min_df = 3, max_df = .7, sublinear_tf=True)

# Transform the training data
tfidf_train = tfidf_vectorizer.fit_transform(X_train)

# Transform the test data
tfidf_test = tfidf_vectorizer.transform(X_test)


# Create Vocab with top words and tokenize again

In [ ]:
# It reduced test accuracy back to 43% and training went from 50% to 44%

tfidf_vectorizer = TfidfVectorizer(vocabulary=top_100_ls, stop_words=my_stop_words, min_df = 3, max_df = .7, sublinear_tf=True)

# Transform the training data
tfidf_train = tfidf_vectorizer.fit_transform(X_train)

# Transform the test data
tfidf_test = tfidf_vectorizer.transform(X_test)


# Run Naive Bayes again

In [97]:
# Use Naive Bayes model -----

from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

nb_classifier = MultinomialNB(alpha=.7)

# Fit and check accuracy
nb_classifier.fit(tfidf_train, y_train)
pred = nb_classifier.predict(tfidf_test)

# Classification Report

In [104]:
# Create classification report taken from here: https://towardsdatascience.com/multi-class-text-classification-model-comparison-and-selection-5eb066197568
from sklearn.metrics import classification_report

print('Test')
class_labels = nb_classifier.classes_
print(classification_report(y_test, pred,target_names=class_labels))

print('Training')
pred_x = nb_classifier.predict(tfidf_train)
print(classification_report(y_train, pred_x,target_names=class_labels))


Test
              precision    recall  f1-score   support

       99222       0.49      0.59      0.54      2541
       99223       0.61      0.34      0.44      2611
       99254       0.69      0.56      0.62      2606
       99291       0.47      0.66      0.55      2618

    accuracy                           0.54     10376
   macro avg       0.56      0.54      0.54     10376
weighted avg       0.56      0.54      0.54     10376

Training
              precision    recall  f1-score   support

       99222       0.53      0.62      0.57      5319
       99223       0.68      0.41      0.51      5249
       99254       0.72      0.61      0.66      5254
       99291       0.51      0.70      0.59      5242

    accuracy                           0.58     21064
   macro avg       0.61      0.58      0.58     21064
weighted avg       0.61      0.58      0.58     21064



In [99]:
metrics.accuracy_score(y_test, pred)

# """
# V1 NLP Model Accuracy: 0.117
# Wow, I've got a long way to go to improve accuracy
# V2 NLP Model Accuracy: 0.14
# V3 NLP Model Accuracy: .40
# """

# Confusion matrix 
# confusion_mtrx = metrics.confusion_matrix(y_test.astype(str), pred) # 1380, 1380
# confusion_mtrx

0.44368476229545556

In [39]:
# Logistical Model accuracy
metrics.accuracy_score(y_test, logist_pred)
# .39



0.3560782168740599

# Vectorize Test

In [84]:
from sklearn.feature_extraction.text import CountVectorizer

vocab = ['love', 'happy', 'run']
count_vectorizer = CountVectorizer(vocabulary = vocab)
x = count_vectorizer.fit_transform(['happy', 'run', 'run', 'run'])
print(x)

  (0, 1)	1
  (1, 2)	1
  (2, 2)	1
  (3, 2)	1
